# RoBERTa

In [ ]:
!pip install syllapy
!pip install ety

## Imports

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import nltk
from collections import Counter
import syllapy
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
import ety
from tqdm import tqdm

import tensorflow as tf
from transformers import (
    BertTokenizer,
    TFBertModel,
    XLMRobertaTokenizer,
    TFXLMRobertaModel,
)
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import (
    Embedding,
    Bidirectional,
    LSTM,
    Dense,
    Dropout,
    Input,
    Flatten,
    concatenate,
    BatchNormalization,
)
from keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model

tqdm.pandas()
nltk.download("wordnet")
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

## Read and Process Data

Below, I pre-process the text data along with performing feature extraction.  The following code takes text input, performs various operations on it, and returns a set of linguistic statistics and processed data.

The following code:

- Converts the input text to lowercase
- Tokenizes the text into sentences using the Natural Language Toolkit (`nltk`), and each sentence is further tokenized into words. POS tags are assigned to each word.
- For each word in the text, the code does the following:
  - Determines the word's origin language (etymology).
  - Retrieves the full name of the POS tag from the `pos_mapping`.
  - Counts the number of syllables in the word.
  - Calculates the length of the word.
- Calculates various linguistic statistics, such as the mean syllable count, the number of sentences, the mean sentence length, the mean word length, and the total number of words.

In [ ]:
df = pd.read_csv(
    "/kaggle/input/clear-corpus-6-01-clear-corpus-6-01/CLEAR Corpus 6.01 - CLEAR Corpus 6.01.csv"
)

In [ ]:
pos_mapping = {
    "CC": "Coordinating Conjunction",
    "CD": "Cardinal Digit",
    "DT": "Determiner",
    "EX": "Existential There",
    "FW": "Foreign Word",
    "IN": "Preposition or Subordinating Conjunction",
    "JJ": "Adjective",
    "JJR": "Adjective, Comparative",
    "JJS": "Adjective, Superlative",
    "LS": "List Item Marker",
    "MD": "Modal",
    "NN": "Noun, Singular or Mass",
    "NNS": "Noun, Plural",
    "NNP": "Proper Noun, Singular",
    "NNPS": "Proper Noun, Plural",
    "PDT": "Predeterminer",
    "POS": "Possessive Ending",
    "PRP": "Personal Pronoun",
    "PRP$": "Possessive Pronoun",
    "RB": "Adverb",
    "RBR": "Adverb, Comparative",
    "RBS": "Adverb, Superlative",
    "RP": "Particle",
    "TO": "to",
    "UH": "Interjection",
    "VB": "Verb, Base Form",
    "VBD": "Verb, Past Tense",
    "VBG": "Verb, Gerund or Present Participle",
    "VBN": "Verb, Past Participle",
    "VBP": "Verb, Non-3rd Person Singular Present",
    "VBZ": "Verb, 3rd Person Singular Present",
    "WDT": "Wh-determiner",
    "WP": "Wh-pronoun",
    "WP$": "Possessive Wh-pronoun",
    "WRB": "Wh-adverb",
}


def process_text(text):
    text = text.lower()

    word_origins = []
    word_pos = []
    syllable_counts = []
    sentence_lengths = []
    word_lengths = []

    sentences = sent_tokenize(text)

    for sentence in sentences:
        tokens = nltk.word_tokenize(sentence)
        pos_tags = nltk.pos_tag(tokens)
        sentence_lengths.append(len(pos_tags))
        for token, pos in pos_tags:
            origin = ety.origins(token)
            if origin:
                origin = origin[0].language.name
            else:
                origin = "unknown"
            word_origins.append(origin)
            full_pos_name = pos_mapping.get(pos, pos)
            word_pos.append(full_pos_name)
            syllables = syllapy.count(token)
            syllable_counts.append(syllables)
            word_lengths.append(len(token))

    processed_excerpt = text
    origin_counts = Counter(word_origins)
    pos_counts = Counter(word_pos)
    mean_syllable_count = np.mean(syllable_counts)
    num_sentences = len(sentences)
    mean_sentence_length = np.mean(sentence_lengths)
    num_words = np.sum(sentence_lengths)
    mean_word_length = np.mean(word_lengths)

    return (
        word_origins,
        origin_counts,
        word_pos,
        pos_counts,
        syllable_counts,
        mean_syllable_count,
        num_sentences,
        mean_sentence_length,
        mean_word_length,
        num_words,
        processed_excerpt,
    )

In [ ]:
df[
    [
        "word_origins",
        "word_origin_counts",
        "pos",
        "pos_counts",
        "syllable_counts",
        "mean_syllable_count",
        "num_sentences",
        "mean_sentence_length",
        "mean_word_length",
        "num_words",
        "processed_excerpt",
    ]
] = df["Excerpt"].progress_apply(lambda x: pd.Series(process_text(x)))

## RoBERTa Model

Below, I prepare text data for the machine learning model.

- **RoBERTa Fine Tuning:** Set the last layer of RoBERTa to be trainable, so we can fine tune it.
- **Fit Tokenizer:** The Tokenizer is trained on preprocessed text excerpts. This step helps the Tokenizer build a vocabulary and assign a unique integer index to each word in the text.
- **Convert Text to Sequences:** The text sequences in the DataFrame are converted into sequences of integers using the Tokenizer
- **Pad Sequences:** To ensure all sequences have the same length, I pad them with zeros to a maximum length.
- **Attention Masks:** Apply attention masks.
- **Label Creation:** Labels for the data are created from the "BT Easiness" column.
- **Data Splitting:** The data is split into training and testing sets. I allocate 20% of the data for testing.
- **Early Stopping:** I define early stopping criteria for model training.

In [ ]:
max_sequence_length = 400
labels = df["BT Easiness"]

xlm_roberta_model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(xlm_roberta_model_name)
xlm_roberta_model = TFXLMRobertaModel.from_pretrained(xlm_roberta_model_name)

# Set XLM-RoBERTa layers to non-trainable except the last one
for layer in xlm_roberta_model.layers:
    layer.trainable = False
for layer in xlm_roberta_model.layers[-1:]:
    layer.trainable = True

# Tokenization and data preparation using XLM-RoBERTa
input_ids = []
attention_mask = []
for text in df["processed_excerpt"]:
    tokens = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_sequence_length,
        return_tensors="tf",
    )
    input_ids.append(tokens["input_ids"][0])
    attention_mask.append(tokens["attention_mask"][0])

# Split the data into training and testing sets
(
    input_ids_train,
    input_ids_test,
    attention_mask_train,
    attention_mask_test,
    labels_train,
    labels_test,
) = train_test_split(input_ids, attention_mask, labels, test_size=0.2, random_state=42)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices(
    (
        {
            "input_ids": input_ids_train,
            "attention_mask": attention_mask_train,
        },
        labels_train,
    )
).batch(2)
test_dataset = tf.data.Dataset.from_tensor_slices(
    (
        {
            "input_ids": input_ids_test,
            "attention_mask": attention_mask_test,
        },
        labels_test,
    )
).batch(2)

Define and train a model with text inputs

- **Model Definition**
    - Two input layers are defined: `input_ids_layer` and `attention_mask_layer` for processing text data.
    - The XLM-RoBERTa model is used to process the input text data.
    - The output is flattened.
    - A dropout layer is applied to reduce overfitting.
    - Batch Normalization is applied to the dropout output.
    - A dense layer follows
    - Followed by the output layer for regression.
- **Model Training:**
    - The model is trained for 30 epochs with a batch size of 15 and early stopping.

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

# Define input layers
input_ids_layer = Input(shape=(max_sequence_length,), dtype=tf.int32, name="input_ids")
attention_mask_layer = Input(
    shape=(max_sequence_length,),
    dtype=tf.int32,
    name="attention_mask",
)

# XLM-RoBERTa model for text embeddings
xlm_roberta_output = xlm_roberta_model(
    input_ids=input_ids_layer, attention_mask=attention_mask_layer
)[
    0
]  # Use index 0 for the output
xlm_roberta_output = Flatten()(xlm_roberta_output)  # Flatten the XLM-RoBERTa output
dropout = Dropout(0.2, name="xlm_roberta_dropout")(xlm_roberta_output)

merged_bn = BatchNormalization(name="merged_batch_norm")(dropout)
merged_dense = Dense(64, activation="relu", name="merged_dense")(merged_bn)
output = Dense(1, activation="linear", name="output")(merged_dense)

# Create the final model with descriptive layer names
model = Model(inputs=[input_ids_layer, attention_mask_layer], outputs=output)

model.compile(optimizer="adam", loss="mean_squared_error")
model.summary()

history = model.fit(
    train_dataset,
    epochs=30,
    batch_size=50,
    validation_data=test_dataset,
    callbacks=[early_stopping],
)

In [ ]:
best_epoch = np.argmin(history.history['val_loss'])
best_loss = history.history['val_loss'][best_epoch]
print(f"The loss of the best model is: {best_loss}")

Plot the testing and training loss.

In [ ]:
train_loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = list(range(1, len(train_loss) + 1))  # Convert range to list

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=epochs,
        y=train_loss,
        mode="lines",
        name="Training Loss",
        line=dict(color="blue"),
    )
)
fig.add_trace(
    go.Scatter(
        x=epochs,
        y=val_loss,
        mode="lines",
        name="Validation Loss",
        line=dict(color="red"),
    )
)
fig.update_layout(
    title="Training and Validation Loss",
    xaxis_title="Epochs",
    yaxis_title="Loss",
    showlegend=True,
    template="plotly_white",
)
fig.show()